[View in Colaboratory](https://colab.research.google.com/github/arealdeadone/Codes/blob/master/Stars.ipynb)

In [0]:
import numpy as np
import pandas as pd

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('HTRU2_train').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

In [341]:
df_train = pd.DataFrame.from_records(rows, columns=rows[0])

df_train.dropna(how='any')
df_train.head()


,observation_ID,mean_integrated,sd_integrated,kurtosis_integrated,skewness_integrated,mean_dmsnr,sd__dmsnr,kurtosis_dmsnr,skewness_dmsnr,class
0,observation_ID,mean_integrated,sd_integrated,kurtosis_integrated,skewness_integrated,mean_dmsnr,sd__dmsnr,kurtosis_dmsnr,skewness_dmsnr,class
1,AXC-00001,106.7734375,52.00649221,0.214592301,-0.453809418,2.096153846,15.12146867,9.676235451,111.128099,0
2,AXC-00002,119.84375,44.7870722,0.121752316,0.253787505,6.699832776,32.61059444,5.172561838,26.92770145,0
3,AXC-00003,131.1953125,50.06590322,-0.090631313,-0.050356626,3.226588629,18.91018188,7.665352563,70.14302195,0
4,AXC-00004,112.5390625,42.10587517,0.345054075,0.768922856,2.617056856,14.57510282,8.611910393,101.0743835,0


In [342]:
worksheet_test = gc.open('HTRU2_test').sheet1
rows_test = worksheet_test.get_all_values()

df_test = pd.DataFrame.from_records(rows_test, columns=rows_test[0])

df_test.dropna()
df_test.head()

,observation_ID,mean_integrated,sd_integrated,kurtosis_integrated,skewness_integrated,mean_dmsnr,sd__dmsnr,kurtosis_dmsnr,skewness_dmsnr
0,observation_ID,mean_integrated,sd_integrated,kurtosis_integrated,skewness_integrated,mean_dmsnr,sd__dmsnr,kurtosis_dmsnr,skewness_dmsnr
1,AXC-12001,123.09375,51.92895349,0.189234125,-0.271854851,1.732441472,15.44632473,11.14290925,141.8993694
2,AXC-12002,100.890625,39.49628827,0.518331071,1.550456789,1.733277592,12.30941382,12.45360191,204.6668024
3,AXC-12003,137.3515625,52.65951805,0.017980891,-0.45847848,2.198996656,14.07346464,9.866782711,126.6391656
4,AXC-12004,99.6171875,39.39367039,0.653726942,1.323436682,8.113712375,32.54709137,4.482549198,20.87979433


In [343]:
m_train = df_train.shape[0]
print(df_train.shape)
m_train

(12001, 10)


12001

In [344]:
train_data = df_train.iloc[1:9001, 1:9]
output_labels_train = df_train.iloc[1:9001,9:]

test_data = df_train.iloc[9001:, 1:9]
output_labels_test = df_train.iloc[9001:,9:]

test_data_orig = df_test.iloc[1:, 1:]


train_data.head()

,mean_integrated,sd_integrated,kurtosis_integrated,skewness_integrated,mean_dmsnr,sd__dmsnr,kurtosis_dmsnr,skewness_dmsnr
1,106.7734375,52.00649221,0.214592301,-0.453809418,2.096153846,15.12146867,9.676235451,111.128099
2,119.84375,44.7870722,0.121752316,0.253787505,6.699832776,32.61059444,5.172561838,26.92770145
3,131.1953125,50.06590322,-0.090631313,-0.050356626,3.226588629,18.91018188,7.665352563,70.14302195
4,112.5390625,42.10587517,0.345054075,0.768922856,2.617056856,14.57510282,8.611910393,101.0743835
5,117.2890625,46.04942188,0.339171469,0.425746936,2.5,20.91464178,9.001622403,83.27324249


In [345]:
train_data_flattened = train_data.T
output_lables_train_flattened = output_labels_train.T

test_data_flattened = test_data.T
output_lables_test_flattened = output_labels_test.T

print(train_data_flattened.shape)
print(test_data_flattened.shape)

(8, 9000)
(8, 3000)


In [346]:
print(output_lables_train_flattened.shape)
print(output_lables_test_flattened.shape)

(1, 9000)
(1, 3000)


In [0]:
def sigmoid(z):
    s = 1/(1+np.exp(-1*z))
    
    return s

In [0]:
def initialize_with_zeros(dim):
    
    w = np.zeros((dim,1))
    b = 0
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [349]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

w = [[0.]
 [0.]]
b = 0


In [0]:
def propagate(w, b, X, Y):

    m = X.shape[1]
    
    A = sigmoid(np.dot(w.T,X)+b)
    cost = -np.sum(Y*np.log(A)+(1-Y.T).T*np.log((1-A.T).T))/m 

    dw = np.dot(X,(A-Y).T) / m
    db = np.sum(A-Y) / m

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [351]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[0.99845601]
 [2.39507239]]
db = 0.001455578136784208
cost = 5.801545319394553


In [0]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        
        
        grads, cost = propagate(w,b,X,Y)
       
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate*dw
        b = b - learning_rate*db
       
        if i % 100 == 0:
            costs.append(cost)
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [353]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

w = [[0.19033591]
 [0.12259159]]
b = 1.9253598300845747
dw = [[0.67752042]
 [1.41625495]]
db = 0.21919450454067657


In [0]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T,X)+b)
    
    for i in range(A.shape[1]):
        
        if A[0][i] <= 0.5:
            Y_prediction[0][i] = 0
        else:
            Y_prediction[0][i] = 1
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [355]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[1. 1. 0.]]


In [0]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w,b,X_train,Y_train,num_iterations,learning_rate,print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w,b,X_test)
    Y_prediction_train = predict(w,b,X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [357]:
w,b = initialize_with_zeros(test_data_flattened.shape[0])
print("w = "+str(w))
print("b = "+str(b))
# np.dot(w.T,test_data_flattened.as_matrix())
# pd.to_numeric(test_data_flattened.iloc[0,0:])
test_data_flattened_num = [pd.to_numeric(test_data_flattened.values[i]) for i in range(test_data_flattened.shape[0])]
test_data_flattened_num = np.array(test_data_flattened_num)
np.dot(w.T,test_data_flattened_num)

w = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
b = 0


array([[0., 0., 0., ..., 0., 0., 0.]])

In [358]:
test_data_x = np.array([pd.to_numeric(test_data_flattened.values[i]) for i in range(test_data_flattened.shape[0])])
train_data_x = np.array([pd.to_numeric(train_data_flattened.values[i]) for i in range(train_data_flattened.shape[0])])
output_lables_train_y = np.array([pd.to_numeric(output_lables_train_flattened.values[i]) for i in range(output_lables_train_flattened.shape[0])])
output_lables_test_y = np.array([pd.to_numeric(output_lables_test_flattened.values[i]) for i in range(output_lables_test_flattened.shape[0])])
test_data_x


array([[ 1.38429688e+02,  2.17421875e+01,  8.71640625e+01, ...,
         8.42500000e+01,  8.75546875e+01,  8.77421875e+01],
       [ 5.23973228e+01,  2.93633610e+01,  0.00000000e+00, ...,
         3.78846935e+01,  3.89909462e+01,  4.73133250e+01],
       [-1.19862634e-01,  6.29840751e+00,  5.09886945e-01, ...,
         6.83532319e-01,  6.12620871e-01,  3.69248123e-01],
       ...,
       [ 1.80735476e+01,  7.90060862e+01,  2.67713737e+01, ...,
         1.27992345e+01,  1.88431543e+01,  2.19727322e+01],
       [ 8.99225286e+00,  5.01918895e-01,  4.56881002e+00, ...,
         1.08951479e+01,  8.24427736e+00,  7.79313139e+00],
       [ 9.70320632e+01, -8.70267085e-01,  2.21488681e+01, ...,
         1.58267737e+02,  7.55630475e+01,  6.33409752e+01]])

In [359]:
d = model(train_data_x, output_lables_train_y, test_data_x, output_lables_test_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.118324
Cost after iteration 200: 0.111605
Cost after iteration 300: 0.107888
Cost after iteration 400: 0.105398
Cost after iteration 500: 0.103584
Cost after iteration 600: 0.102193
Cost after iteration 700: 0.101087
Cost after iteration 800: 0.100182
Cost after iteration 900: 0.099427
Cost after iteration 1000: 0.098785
Cost after iteration 1100: 0.098232
Cost after iteration 1200: 0.097749
Cost after iteration 1300: 0.097323
Cost after iteration 1400: 0.096945
Cost after iteration 1500: 0.096605
Cost after iteration 1600: 0.096298
Cost after iteration 1700: 0.096020
Cost after iteration 1800: 0.095766
Cost after iteration 1900: 0.095532
train accuracy: 97.25555555555556 %
test accuracy: 97.43333333333334 %


In [360]:
actual_pred = test_data_orig.T
actual_pred

,1,2,3,4,5,6,7,8,9,10,...,5889,5890,5891,5892,5893,5894,5895,5896,5897,5898
mean_integrated,123.09375,100.890625,137.3515625,99.6171875,115.1796875,113.1875,141.3203125,114.390625,108.8984375,115.578125,...,108.765625,113.4296875,131.4765625,114.8359375,133.1875,126.4375,105.5546875,134.890625,96.421875,135.59375
sd_integrated,51.92895349,39.49628827,52.65951805,39.39367039,46.4031979,57.8435052,55.86478285,35.82863606,51.71025645,45.82040508,...,59.39821709,53.73178232,52.16020466,53.54186116,46.65661629,47.27855453,49.28469726,55.07781915,48.30625111,51.98293553
kurtosis_integrated,0.189234125,0.518331071,0.017980891,0.653726942,0.150304307,0.693296624,-0.257080818,0.453346891,0.073618355,0.383154047,...,0.129117577,0.198426986,0.048921053,0.275238199,-0.326708259,0.249057379,0.217802083,-0.139759422,0.243269235,-0.006378839
skewness_integrated,-0.271854851,1.550456789,-0.45847848,1.323436682,-0.022155432,-0.1424789,-0.718214496,1.580612173,-0.432564868,0.266932974,...,-0.734403656,-0.225134084,-0.047418166,-0.282135352,0.156832374,0.181704989,-0.169812357,-0.628622776,-0.179898575,-0.247849052
mean_dmsnr,1.732441472,1.733277592,2.198996656,8.113712375,3.201505017,4.668060201,2.884615385,1.893812709,108.9456522,7.191471572,...,10.44899666,15.34866221,3.072742475,6.621237458,2.732441472,0.701505017,5.602842809,3.284280936,6.825250836,2.77006689
sd__dmsnr,15.44632473,12.30941382,14.07346464,32.54709137,17.52376402,25.28329819,24.15263055,12.90541596,78.42685847,31.44543441,...,39.57966743,50.42663806,17.51890215,33.46509464,16.51279987,11.26847933,28.69174118,17.98195269,34.14449521,21.4260369
kurtosis_dmsnr,11.14290925,12.45360191,9.866782711,4.482549198,7.239112622,6.844521764,8.529005068,11.07042787,-0.615833144,4.817609566,...,4.149145439,3.225795188,8.448585737,5.277951757,8.452670652,18.34625198,5.676588354,8.347579405,5.250675546,8.20557689
skewness_dmsnr,141.8993694,204.6668024,126.6391656,20.87979433,64.57221524,50.69205775,71.89934906,165.3345208,-1.496695207,23.61960298,...,16.95093355,9.137997165,88.30312381,27.2734502,88.05981307,358.056205,33.47001918,88.10811859,27.3930542,69.36548173


In [361]:
actual_pred_numeric = np.array([pd.to_numeric(actual_pred.values[i]) for i in range(actual_pred.shape[0])])
actual_pred_numeric

array([[ 1.23093750e+02,  1.00890625e+02,  1.37351562e+02, ...,
         1.34890625e+02,  9.64218750e+01,  1.35593750e+02],
       [ 5.19289535e+01,  3.94962883e+01,  5.26595181e+01, ...,
         5.50778192e+01,  4.83062511e+01,  5.19829355e+01],
       [ 1.89234125e-01,  5.18331071e-01,  1.79808910e-02, ...,
        -1.39759422e-01,  2.43269235e-01, -6.37883900e-03],
       ...,
       [ 1.54463247e+01,  1.23094138e+01,  1.40734646e+01, ...,
         1.79819527e+01,  3.41444952e+01,  2.14260369e+01],
       [ 1.11429093e+01,  1.24536019e+01,  9.86678271e+00, ...,
         8.34757940e+00,  5.25067555e+00,  8.20557689e+00],
       [ 1.41899369e+02,  2.04666802e+02,  1.26639166e+02, ...,
         8.81081186e+01,  2.73930542e+01,  6.93654817e+01]])

In [364]:
print(d["w"])
print(d["b"])
labels = predict(d["w"],d["b"], actual_pred_numeric)
labels = labels.T

[[-0.03235179]
 [-0.01969771]
 [ 0.17686494]
 [ 0.36945943]
 [-0.00395394]
 [ 0.0386805 ]
 [-0.03504593]
 [-0.00294882]]
-0.0003345942105387511


In [0]:
final = pd.DataFrame.from_dict(labels)
observation_ID = df_test.iloc[1:,0]
final['observation_ID'] = observation_ID.values
final.columns = ['class', 'observation_ID']
final = final[['observation_ID','class']]
final.head()

final.to_csv('Output.csv', sep=',')

# final = pd.DataFrame({'observation_ID': observation_ID.values, 'class': labels})

# final.head()

In [378]:
!ls


adc.json  datalab  Output.csv  sample_data


In [0]:
from google.colab import files

files.download('Output.csv')